In [1]:
! pip install -U segmentation-models

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 1.4 MB/s eta 0:00:00


In [51]:
! pip install --upgrade keras tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 3.5 MB/s eta 0:00:00a 0:00:010m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 1.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 65.5 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 41.2 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.2/439.2 kB 4.7 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 29.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.6/124.6 kB 13.3 MB/s eta 0:00:00
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 1.12
    Uninstalling flatbuffers-1.12:
      Successfully uninstalled flatbuffers-1.12
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.6.0
    Uninstalling tensorflow-estimator-2.6.0:
      Successfully uninstalled tensorflow-estimator-2.6.0
  Attempti

In [2]:
import tensorflow as tf
import keras
import random

In [3]:
import segmentation_models as sm
sm.set_framework('tf.keras')
sm.framework()
from segmentation_models import Unet

Segmentation Models: using `keras` framework.


In [4]:
from segmentation_models import get_preprocessing
from segmentation_models.losses import bce_jaccard_loss
from segmentation_models.metrics import iou_score

In [5]:
input_files = []
with open('/kaggle/input/wildfire-segmentation-dataset/classes/mixed.txt') as f:
    input_files = f.read().splitlines()
random.shuffle(input_files)

In [6]:
from sklearn.model_selection import train_test_split
train_files, valid_files = train_test_split(input_files, test_size=0.2, random_state=101)

In [7]:
import numpy as np
import keras
from tensorflow.keras.preprocessing.image import load_img
from keras.utils.np_utils import to_categorical   
import tensorflow as tf

In [8]:
class Sequence_generator(tf.keras.utils.Sequence):
    """Helper to iterate over the data (as Numpy arrays)."""

    def __init__(self, batch_size, img_size, input_files, bands, input_dir, mask_dir):
        self.batch_size = batch_size
        self.img_size = img_size
        self.input_files = input_files
        self.bands = bands
        self.input_dir = input_dir
        self.mask_dir = mask_dir

    def __len__(self):
        return len(self.input_files) // self.batch_size

    def __getitem__(self, idx):
        """Returns tuple (input, target) correspond to batch #idx."""
        i = idx * self.batch_size
        batch_files = self.input_files[i:i+self.batch_size]
        x = np.zeros((self.batch_size,) + self.img_size + (len(self.bands),), dtype="float32")
        for j, file_name in enumerate(batch_files):
            for b, band in enumerate(self.bands):
                img = load_img(self.input_dir + f'{band}/{file_name}', target_size=self.img_size, color_mode='grayscale')
                x[j,:,:,b] = img
#         x = x.astype('uint8')
        y = np.zeros((self.batch_size,) + self.img_size + (3,), dtype="float32")
        for j, file_name in enumerate(batch_files):
            img = load_img(self.mask_dir + file_name, target_size=self.img_size, color_mode='grayscale')
            y[j] = to_categorical(img, num_classes=3)
#             y[j] = img
        return x, y
    def on_epoch_end(self):
        random.shuffle(self.input_files)
        return super().on_epoch_end()

In [9]:
unet_bands = ['B12', 'B11', 'B8A']

In [34]:
batch_size = 16
epochs = 40
image_size = (256,256)
input_dir =  "/kaggle/input/wildfire-segmentation-dataset/classes/mixed/"
mask_dir = "/kaggle/input/wildfire-segmentation-dataset/classes/mixed/normalized_mask/"

In [35]:
train_gen = Sequence_generator(batch_size, image_size, train_files, unet_bands, input_dir, mask_dir)
valid_gen = Sequence_generator(batch_size, image_size, valid_files, unet_bands, input_dir, mask_dir)

In [36]:
t1 = train_gen.__getitem__(0)
t1[1].shape

(16, 256, 256, 3)

In [48]:
# models = sm.Unet()
# model = Unet('resnet34', encoder_weights='imagenet')
# model = Unet('resnet34', classes=3, activation='softmax')
model = Unet('seresnet34',classes=3, input_shape=image_size+(3,), encoder_weights='imagenet')

86327296/86315168 [==============================] - 4s 0us/step


In [50]:
save_file = 'unet_seresnet34_B12_B11_B8A.h5'

In [44]:
callbacks = [
    keras.callbacks.ModelCheckpoint(save_file, save_best_only=True)
]

model.compile('adamax', loss=bce_jaccard_loss, metrics=[iou_score])

# fit model
model.fit(
    train_gen,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=valid_gen,
    callbacks=callbacks
)

Epoch 1/40
53/53 [==============================] - 57s 554ms/step - loss: 1.0700 - iou_score: 0.3952 - val_loss: 2.1666 - val_iou_score: 0.3360
Epoch 2/40
53/53 [==============================] - 24s 455ms/step - loss: 0.8423 - iou_score: 0.5167 - val_loss: 1.1919 - val_iou_score: 0.4141
Epoch 3/40
53/53 [==============================] - 24s 456ms/step - loss: 0.7685 - iou_score: 0.5633 - val_loss: 1.2270 - val_iou_score: 0.5067
Epoch 4/40
53/53 [==============================] - 25s 462ms/step - loss: 0.7081 - iou_score: 0.5968 - val_loss: 1.0179 - val_iou_score: 0.5112
Epoch 5/40
53/53 [==============================] - 24s 458ms/step - loss: 0.6879 - iou_score: 0.6130 - val_loss: 0.7945 - val_iou_score: 0.5758
Epoch 6/40
53/53 [==============================] - 24s 453ms/step - loss: 0.6306 - iou_score: 0.6412 - val_loss: 0.7708 - val_iou_score: 0.6536
Epoch 7/40
53/53 [==============================] - 24s 458ms/step - loss: 0.5541 - iou_score: 0.6873 - val_loss: 0.7134 - val_iou

In [46]:

model.save(save_file)

In [47]:
from IPython.display import FileLink
FileLink(save_file)

/kaggle/working/unet_resnext50_model_9_B12_B11_B8A.h5